# 6교시 Spark JDBC to MySQL

### 목차
* [1. MySQL 접속 예제](#1.-MySQL-접속-예제)
* [10. 참고자료](#10.-참고자료)


In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

21/09/05 04:03:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 1. MySQL 접속 예제

In [3]:
this_style_also_work_but = """
from pyspark.sql.context import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc)
seoul_popular_trip = (
    sqlContext.read.format("jdbc")
    .option("url", "jdbc:mysql://mysql:3306/testdb")
    .option("driver", "com.mysql.jdbc.Driver")
    .option("dbtable", "seoul_popular_trip")
    .option("user", "sqoop")
    .option("password", "sqoop")
    .load()
)
"""

In [4]:
connString = "jdbc:mysql://mysql:3306/testdb"
tableName = "seoul_popular_trip"
accessInfo = {"user":"sqoop", "password":"sqoop"}

In [5]:
seoul_popular_trip = spark.read.jdbc(connString, tableName, properties=accessInfo)

In [6]:
seoul_popular_trip.printSchema()
display(seoul_popular_trip)

root
 |-- category: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- naddress: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- tag: string (nullable = true)



category,id,name,address,naddress,tel,tag
0,281,통인시장,110-043 서울 종로구 통인동 10-3,03036 서울 종로구 자하문로15길 18,02-722-0911,"엽전도시락,종로통인시장,통인시장닭꼬치,런닝맨,엽전시장,통인시장데이트,효자베이커리,통인시장, 1박2일,기름떡볶이"
0,345,타르틴,140-863 서울 용산구 이태원동 119-15,04350 서울 용산구 이태원로23길 4 (이태원동),02-3785-3400,"타르틴,이태원디저트카페,파이,런닝맨,파이맛집,이태원맛집, 유재석,식신로드,타르트맛집"
0,383,해랑,135-827 서울 강남구 논현동 183,06120 서울 강남구 강남대로118길 12 (부경빌딩),02-544-7270,"해물탕,해물음식점,해물요리,강남맛집,논현동음식점,강남,신논현맛집,해랑"
0,413,인도로 가는 길,110-300 서울 종로구 관훈동 198-36,"03149 서울 종로구 인사동5길 20 (관훈동, 오원빌딩)",02-738-7773,"인도 요리,인도레스토랑,인도로 가는 길,광화문,할랄레스토랑,인도음식점,인사동음식점,커리음식점"
0,420,올데이 브런치 카페 B Meal(비밀),140-893 서울 용산구 한남동 738-7,04348 서울 용산구 이태원로55가길 35,02-797-0715,"이태원브런치,브런치음식점,올데이 브런치 카페 B Meal,카페,한남동음식점,이태원,이태원음식점,한남동카페,이태원카페"
0,428,장이오,137-857 서울 서초구 서초동 1321-9,06619 서울 서초구 강남대로55길 24 (풍림아이원매직),02-3473-1025,"장이오,강남,한식음식점,한식전문점,강남역음식점,된장찌개,된장,서초동음식점, 간장, 고추장"
0,429,진진바라 (서울역점),140-709 서울 용산구 동자동 12,04323 서울 용산구 후암로 107 (게이트웨이타워),02-777-8004,"진진바라,서울역음식점,용산음식점,한식음식점,서울역,한정식,한식전문점"
0,441,울프강 스테이크 하우스,135-954 서울 강남구 청담동 89-6,"06016 서울 강남구 선릉로152길 21 (청담동, 영인빌딩)",02-556-8700,"뉴욕3대스테이크,스테이크,스테이크음식점,청담동음식점,강남,레스토랑,울프강 스테이크 하우스"
0,446,난향,120-834 서울 서대문구 창천동 72-21,03787 서울 서대문구 신촌로 67 (거촌빌딩),02-322-6900,"중화요리,중국집,난향,신촌음식점,중국 음식점,고급중식당,중식레스토랑"
0,463,논현삼계탕(명동점),100-860 서울 중구 충무로2가 7-1,04537 서울 중구 명동10길 41,02-518-4602,"논현삼계탕,명동음식점,삼계탕전문점,명동,백숙,보양식,삼계탕"


In [8]:
from pyspark.sql.functions import *
# seoul_popular_trip.limit(5).select("id", "name", explode(split("tag", ","))).groupBy("name").count().show()
top10 = (
    seoul_popular_trip
    .select("id", "name", explode(split("tag", ",")))
    .groupBy("name")
    .count()
    .orderBy(desc("count"))
    .limit(10)
)

In [9]:
display(top10)

name,count
CJ 올리브마켓,36
더 스트리트 (THE STREET),32
통인시장,30
신세계면세점 (명동점),28
신세계백화점 본점,27
401 (홍대),26
젠틀몬스터 신사 플래그쉽 스토어,26
장진우식당,26
설빙 신림2호점,24
리아네이처 (Lyanature),22


In [10]:
this_style_also_work_but = """(
    top10.write.format("jdbc")
    .option("url", "jdbc:mysql://mysql:3306/testdb")
    .option("driver", "com.mysql.jdbc.Driver")
    .option("dbtable", "top10")
    .option("user", "sqoop")
    .option("password", "sqoop")
    .mode("overwrite")
    .save()
)"""

In [11]:
top10.write.mode("overwrite").jdbc(connString, "top10", properties=accessInfo)

#### 2.1. 저장된 데이터 확인
```bash
bash> docker-compose exec mysql mysql -usqoop -psqoop
mysql> use testdb;
mysql> select * from top10 order by count desc limit 10;
```

```text
+-----------------------------------------------+-------+
| name                                          | count |
+-----------------------------------------------+-------+
| CJ 올리브마켓                                 |    36 |
| 더 스트리트 (THE STREET)                      |    32 |
| 통인시장                                      |    30 |
| 신세계면세점 (명동점)                         |    28 |
| 신세계백화점 본점                             |    27 |
| 젠틀몬스터 신사 플래그쉽 스토어               |    26 |
| 401 (홍대)                                    |    26 |
| 장진우식당                                    |    26 |
| 설빙 신림2호점                                |    24 |
| 너드온어스 (Nerd on earth)                    |    22 |
+-----------------------------------------------+-------+
```

### <font color=red>1. [고급]</font> `seoul_popular_trip` 테이블에서 
#### 1. 동일한 전화번호(tel)로 가장 많이 등록된 전화번호를 찾으세요
#### 2. 가장 많이 등록된 순서 상위 10개의 전화번호를 추출해주세요
* 전화번호가 null 혹은 empty('')인 값인 경우는 제외해 주세요

#### 3. 데이터베이스에 접속하여 `fraud10` 테이블을 생성해 주세요
* `testdb.fraud10` (`tel` string, `count` int) 이며 직접 생성해 두어야 합니다

#### 4. 해당 데이터를 `fraud10` 테이블에 저장해 주세요

<details><summary>[실습1] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```bash
docker-compose exec mysql mysql -usqoop -psqoop
```

```sql
use testdb;
create table if not exists fraud10 (tel varchar(30), count int);
```

```python
seoul_popular_trip.printSchema()
fraud_candidate = seoul_popular_trip.groupBy("tel").count().orderBy(desc("count")).limit(10)
fraud_numbers = fraud_candidate.na.drop(subset=["tel"]).where("tel != ''")
display(fraud_numbers)
(
    fraud_numbers
    .write.format("jdbc")
    .option("url", "jdbc:mysql://mysql:3306/testdb")
    .option("driver", "com.mysql.jdbc.Driver")
    .option("dbtable", "fraud10")
    .option("user", "sqoop")
    .option("password", "sqoop")
    .mode("overwrite")
    .save()
)```

</details>


In [7]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
# docker-compose exec mysql mysql -usqoop -psqoop
# use testdb;
# create table if not exists fraud10 (tel varchar(30), count int);


root
 |-- category: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- naddress: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- tag: string (nullable = true)



tel,count
02-120,39
+82-2-120,4
02-2077-9000,3
1588-1234,3
02-722-0911,3
02-2631-9368,3
02-771-2500,3
02-325-0805,3
02-739-8288,3


### <font color=red>2. [고급]</font> '1. 고급'에서 저장한 `fraud10` 테이블에서 
#### 1. 스키마를 출력하세요
#### 2. 데이터를 10건 출력하세요

<details><summary>[실습2] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
from pyspark.sql.context import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc)
fraud10_loaded = (
    sqlContext.read.format("jdbc")
    .option("url", "jdbc:mysql://mysql:3306/testdb")
    .option("driver", "com.mysql.jdbc.Driver")
    .option("dbtable", "fraud10")
    .option("user", "sqoop")
    .option("password", "sqoop")
    .load()
)
fraud10_loaded.printSchema()
display(fraud10_loaded)
```

</details>


In [8]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- tel: string (nullable = true)
 |-- count: long (nullable = true)



tel,count
02-120,39
+82-2-120,4
02-2077-9000,3
1588-1234,3
02-722-0911,3
02-2631-9368,3
02-771-2500,3
02-325-0805,3
02-739-8288,3


## 10. 참고자료

#### 1. [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
#### 2. [PySpark SQL Modules Documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
#### 3. <a href="https://spark.apache.org/docs/3.0.1/api/sql/" target="_blank">PySpark 3.0.1 Builtin Functions</a>
#### 4. [PySpark Search](https://spark.apache.org/docs/latest/api/python/search.html)
#### 5. [Pyspark Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?#module-pyspark.sql.functions)
#### 6. [JDBC To Other Databases](https://spark.apache.org/docs/3.0.1/sql-data-sources-jdbc.html)
